In [ ]:
from matplotlib import pyplot as plt
import numpy as np

## Neural Network Implementation

Define several constants

- `Nl` The total number of odorants

- `Nr` The number of receptors

- `K` The average number of odorants in the mixture

- `s` The probability that an odorant binds to a receptor


In [ ]:
Nl = 20
Nr = 5
K = 5
s = 0.2

Generate values for matrix **S** (receptor sensitivity to odorants)

- Dimension for `S` is `Nr` by `Nl`.

- Probability of an entry to be non-zero is `s`

- The non-zero entries are log-uniformly distributed between $10^{-1}$ and $10^1$

In [ ]:
S = np.zeros((Nr, Nl), dtype=np.float64)
random_arr = np.random.rand(Nr, Nl)
for i in range(S.shape[0]):
    for j in range(S.shape[1]):
        if np.random()